In [7]:
import numpy as np
import pandas as pd
import altair as alt

from tqdm import tqdm

tqdm.pandas()
pd.options.display.max_columns = None
pd.options.display.max_rows = 100
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

In [8]:
from clickhouse_driver import Client as Clickhouse

def click_query(q, **kw):
    click = Clickhouse("localhost")
    return click.query_dataframe(q, params=kw)

In [9]:
click_query("DESCRIBE TABLE obs_web")

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,measurement_uid,String,,,,,
1,input,Nullable(String),,,,,
2,report_id,String,,,,,
3,measurement_start_time,DateTime64(6),,,,,
4,software_name,String,,,,,
5,software_version,String,,,,,
6,test_name,String,,,,,
7,test_version,String,,,,,
8,probe_asn,Int32,,,,,
9,probe_cc,String,,,,,


In [10]:
click_query("DESCRIBE TABLE obs_web_ctrl")

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
0,measurement_uid,String,,,,,
1,input,Nullable(String),,,,,
2,report_id,String,,,,,
3,measurement_start_time,DateTime64(6),,,,,
4,software_name,String,,,,,
5,software_version,String,,,,,
6,test_name,String,,,,,
7,test_version,String,,,,,
8,hostname,String,,,,,
9,observation_id,String,,,,,


In [49]:
MEASUREMENT_START_DAY = '2024-04-01'
MEASUREMENT_END_DAY = '2024-07-01'
ANALYSIS_COUNTRY_CODES = [
    "IN"
]

In [12]:
%%time
df = click_query("""
SELECT 
report_id,
input,
measurement_uid,
probe_cc,
probe_asn,
probe_as_org_name,
probe_as_cc,
network_type,
measurement_start_time,
hostname,
test_name,
ip,
port,
ip_asn,
ip_as_org_name,
resolver_ip,
resolver_cc,
resolver_asn,
resolver_as_org_name,
resolver_as_cc,
dns_engine,
dns_failure,
dns_answer,
dns_answer_type,
http_request_url,
http_failure,
http_runtime,
http_response_body_sha1,
http_response_status_code,
http_response_body_length,
http_response_body_is_truncated,
probe_analysis
FROM obs_web
WHERE probe_cc = 'IN' 
AND measurement_start_time > '2024-04-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'web_connectivity'
""")

CPU times: user 40.5 s, sys: 3.39 s, total: 43.9 s
Wall time: 2min 2s


In [36]:
%%time
target_domains = [
    "www.tiktok.com"
]
df_special = click_query("""
SELECT *
FROM obs_web 
WHERE probe_cc = 'IN' 
AND measurement_start_time > '2023-07-01'
AND measurement_start_time < '2024-07-01'
AND hostname IN %(target_domains)s
""", target_domains=target_domains)

CPU times: user 13.8 s, sys: 918 ms, total: 14.7 s
Wall time: 1min 11s


In [13]:
pd.set_option('display.max_colwidth', None)
df['explorer_url'] = "https://explorer.ooni.org/measurement/" + df['measurement_uid']

In [37]:
pd.set_option('display.max_colwidth', None)
df_special['explorer_url'] = "https://explorer.ooni.org/measurement/" + df_special['report_id'] + "?input=" + df_special['input']

In [38]:
df_special.to_csv('2024-IN-df-special.csv')

In [14]:
df.to_csv('2024-IN-df.csv')

In [6]:
df = pd.read_csv('2024-IN-df.csv')

In [15]:
df[['probe_asn', 'probe_as_org_name']].drop_duplicates(subset=['probe_asn']).to_csv('2024_IN_ASN_Lookup.csv')

In [16]:
len(df)

1863449

In [17]:
df

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,dns_answer_type,http_request_url,http_failure,http_runtime,http_response_body_sha1,http_response_status_code,http_response_body_length,http_response_body_is_truncated,probe_analysis,explorer_url
0,20240401T000455Z_webconnectivity_IN_24309_n1_mt88sqfNmoeZzL2K,https://www.facebook.com/,20240401000458.265234_IN_webconnectivity_6d351b702a66b9e6,IN,24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,wifi,2024-04-01 00:04:57,www.facebook.com,web_connectivity,163.70.144.35,443.0,32934.0,"Facebook, Inc.",125.20.246.123,IN,9498,Bharti Airtel Limited,IN,system,None,163.70.144.35,A,None,None,None,None,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240401000458.265234_IN_webconnectivity_6d351b702a66b9e6
1,20240401T000455Z_webconnectivity_IN_24309_n1_mt88sqfNmoeZzL2K,https://www.facebook.com/,20240401000458.265234_IN_webconnectivity_6d351b702a66b9e6,IN,24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,wifi,2024-04-01 00:04:57,www.facebook.com,web_connectivity,2a03:2880:f188:181:face:b00c:0:25de,443.0,32934.0,"Facebook, Inc.",125.20.246.123,IN,9498,Bharti Airtel Limited,IN,system,None,2a03:2880:f188:181:face:b00c:0:25de,AAAA,None,None,None,None,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240401000458.265234_IN_webconnectivity_6d351b702a66b9e6
2,20240401T000455Z_webconnectivity_IN_24309_n1_mt88sqfNmoeZzL2K,https://www.facebook.com/,20240401000458.265234_IN_webconnectivity_6d351b702a66b9e6,IN,24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,wifi,2024-04-01 00:04:57,www.facebook.com,web_connectivity,None,NaN,NaN,None,125.20.246.123,IN,9498,Bharti Airtel Limited,IN,None,None,None,None,https://www.facebook.com/,None,None,8a6c68513ac82c43ba2f10b8f821d7e5b1f60d1d,200.0,57104.0,0.0,false,https://explorer.ooni.org/measurement/20240401000458.265234_IN_webconnectivity_6d351b702a66b9e6
3,20240401T000455Z_webconnectivity_IN_24309_n1_mt88sqfNmoeZzL2K,https://twitter.com/,20240401000501.878055_IN_webconnectivity_3d03fc0dc28f67e9,IN,24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,wifi,2024-04-01 00:05:00,twitter.com,web_connectivity,104.244.42.193,443.0,13414.0,Twitter Inc.,125.20.246.123,IN,9498,Bharti Airtel Limited,IN,system,None,104.244.42.193,A,None,None,None,None,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240401000501.878055_IN_webconnectivity_3d03fc0dc28f67e9
4,20240401T000455Z_webconnectivity_IN_24309_n1_mt88sqfNmoeZzL2K,https://twitter.com/,20240401000501.878055_IN_webconnectivity_3d03fc0dc28f67e9,IN,24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,wifi,2024-04-01 00:05:00,twitter.com,web_connectivity,None,NaN,NaN,None,125.20.246.123,IN,9498,Bharti Airtel Limited,IN,None,None,None,None,https://twitter.com/,None,None,b80bb1a6fec1ab5062850ef194d6b076263a5f4a,200.0,131072.0,1.0,false,https://explorer.ooni.org/measurement/20240401000501.878055_IN_webconnectivity_3d03fc0dc28f67e9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863444,20240425T125254Z_webconnectivity_IN_133982_n1_8Lh51KHjMoPcruuX,http://aitcofficial.org/,20240425125916.312244_IN_webconnectivity_e8338eefb7e5e1af,IN,133982,Excitel Broadband Private Limited,IN,unknown,2024-04-25 12:59:15,aitcofficial.org,web_connectivity,104.21.60.172,80.0,13335.0,Cloudflare Inc,139.5.252.2,IN,133982,Excitel Broadband Private Limited,IN,system,None,104.21.60.172,A,None,None,None,None,NaN,NaN,NaN,false,https://explorer.ooni.org/measurement/20240425125916.312244_IN_webconnectivity_e8338eefb7e5e1af
1863445,20240425T125254Z_webconnectivity_IN_133982_n1_8Lh51KHjMoPcruuX,http://aitcofficial.org/,20240425125916.312244_IN_webconnectivity_e8338eefb7e5e1af,IN,133982,Excitel Broadband Private Limited,IN,u

In [18]:
df['probe_analysis'].value_counts()

false           1644102
dns               70531
http-failure      57459
tcp_ip            20665
http-diff         20087
Name: probe_analysis, dtype: int64

In [19]:
df['http_response_body_length'].value_counts()

131072.0    86735
162.0       13806
167.0        7200
131071.0     3312
49.0         3148
            ...  
126085.0        1
114095.0        1
24497.0         1
31535.0         1
110763.0        1
Name: http_response_body_length, Length: 21580, dtype: int64

In [20]:
df_dns_answer = df[df['dns_answer_type']=='A'].drop_duplicates(subset=['measurement_uid'])

In [21]:
len(df_dns_answer)

457072

In [22]:
dns_resp_sorted = df_dns_answer[
    (df_dns_answer['probe_analysis'] == 'dns')
].groupby('dns_answer')['hostname'].nunique().sort_values().reset_index()

In [23]:
dns_resp_sorted[
    dns_resp_sorted['hostname'] > 2
]

,dns_answer,hostname
191,82.192.82.226,3
192,69.162.95.3,3
193,74.63.241.19,3
194,82.192.82.227,3
195,70.32.1.32,3
196,37.48.65.152,3
197,185.107.56.195,3
198,192.157.56.142,3
199,185.107.56.53,3
200,162.210.199.65,3


In [24]:
dns_resp_sorted[
    dns_resp_sorted['hostname'] > 10
]

,dns_answer,hostname
222,36.37.242.94,13
223,127.0.0.1,14
224,172.16.16.250,57
225,49.44.79.236,129
226,13.127.247.216,144
227,49.205.171.201,186
228,202.83.21.14,186
229,49.207.46.62,193
230,218.248.112.60,244


In [25]:
dns_suspicious = dns_resp_sorted[dns_resp_sorted['hostname'] > 10]['dns_answer'].values.tolist()
dns_suspicious


['36.37.242.94',
 '127.0.0.1',
 '172.16.16.250',
 '49.44.79.236',
 '13.127.247.216',
 '49.205.171.201',
 '202.83.21.14',
 '49.207.46.62',
 '218.248.112.60']

In [26]:
df_dns_answer[df_dns_answer['dns_answer'].isin(dns_suspicious)].drop_duplicates(subset=['dns_answer']).to_csv('2024_IN_dns_suspicious.csv')

In [27]:
dns_answer_filter = df_dns_answer[df_dns_answer['dns_answer'].isin(dns_suspicious)]

dns_answer_filter = dns_answer_filter.pivot_table(index=['dns_answer', 'probe_cc'], 
              values=['hostname'],
              aggfunc={'hostname': ['nunique', lambda x: ', '.join(set(x))]}).rename(columns = {
                                                                    '<lambda_0>': 'hostnames',
                                                                    'len' : 'hostname_count'}).reset_index()

dns_answer_filter.head()

dns_answer probe_cc  \
                             
0       127.0.0.1       IN   
1  13.127.247.216       IN   
2   172.16.16.250       IN   
3    202.83.21.14       IN   
4  218.248.112.60       IN   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    

In [28]:
dns_answer_filter.to_csv('2024_IN_dns_answer_hostname')

In [29]:
df_http_answer = df[df['http_request_url'].notnull()].drop_duplicates(subset=['measurement_uid'])

In [30]:
df_http_answer.head()

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,dns_answer_type,http_request_url,http_failure,http_runtime,http_response_body_sha1,http_response_status_code,http_response_body_length,http_response_body_is_truncated,probe_analysis,explorer_url
2,20240401T000455Z_webconnectivity_IN_24309_n1_mt88sqfNmoeZzL2K,https://www.facebook.com/,20240401000458.265234_IN_webconnectivity_6d351b702a66b9e6,IN,24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,wifi,2024-04-01 00:04:57,www.facebook.com,web_connectivity,None,NaN,NaN,None,125.20.246.123,IN,9498,Bharti Airtel Limited,IN,None,None,None,None,https://www.facebook.com/,None,None,8a6c68513ac82c43ba2f10b8f821d7e5b1f60d1d,200.0,57104.0,0.0,false,https://explorer.ooni.org/measurement/20240401000458.265234_IN_webconnectivity_6d351b702a66b9e6
4,20240401T000455Z_webconnectivity_IN_24309_n1_mt88sqfNmoeZzL2K,https://twitter.com/,20240401000501.878055_IN_webconnectivity_3d03fc0dc28f67e9,IN,24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,wifi,2024-04-01 00:05:00,twitter.com,web_connectivity,None,NaN,NaN,None,125.20.246.123,IN,9498,Bharti Airtel Limited,IN,None,None,None,None,https://twitter.com/,None,None,b80bb1a6fec1ab5062850ef194d6b076263a5f4a,200.0,131072.0,1.0,false,https://explorer.ooni.org/measurement/20240401000501.878055_IN_webconnectivity_3d03fc0dc28f67e9
7,20240401T000455Z_webconnectivity_IN_24309_n1_mt88sqfNmoeZzL2K,https://www.instagram.com/,20240401000504.810792_IN_webconnectivity_da976597f44c579a,IN,24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,wifi,2024-04-01 00:05:03,www.instagram.com,web_connectivity,None,NaN,NaN,None,125.20.246.123,IN,9498,Bharti Airtel Limited,IN,None,None,None,None,https://www.instagram.com/,None,None,0ace400dd29f27aa4b102cd0d7836ccaa1640228,200.0,131072.0,1.0,false,https://explorer.ooni.org/measurement/20240401000504.810792_IN_webconnectivity_da976597f44c579a
21,20240401T000455Z_webconnectivity_IN_24309_n1_mt88sqfNmoeZzL2K,https://www.youtube.com/,20240401000509.243274_IN_webconnectivity_a017a016b379c7bf,IN,24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,wifi,2024-04-01 00:05:07,www.youtube.com,web_connectivity,None,NaN,NaN,None,125.20.246.123,IN,9498,Bharti Airtel Limited,IN,None,None,None,None,https://www.youtube.com/,None,None,b84b822201a84e58de2501d20a7269e1d9fd2104,200.0,131069.0,1.0,false,https://explorer.ooni.org/measurement/20240401000509.243274_IN_webconnectivity_a017a016b379c7bf
31,20240401T000455Z_webconnectivity_IN_24309_n1_mt88sqfNmoeZzL2K,https://doh-fi.blahdns.com/dns-query?dns=q80BAAABAAAAAAAAA3d3dwdleGFtcGxlA2NvbQAAAQAB,20240401000511.365227_IN_webconnectivity_7f8d974257259198,IN,24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,wifi,2024-04-01 00:05:10,doh-fi.blahdns.com,web_connectivity,None,NaN,NaN,None,125.20.246.123,IN,9498,Bharti Airtel Limited,IN,None,None,None,None,https://doh-fi.blahdns.com/dns-query?dns=q80BAAABAAAAAAAAA3d3dwdleGFtcGxlA2NvbQAAAQAB,None,None,3566908ce23632aacd5b20443cd189d58bffde85,200.0,64.0,0.0,false,https://explorer.ooni.org/measurement/20240401000511.365227_IN_webconnectivity_7f8d974257259198


In [31]:
http_domain_count = df_http_answer[
    df_http_answer['probe_analysis'] == 'http-diff'
].groupby('http_response_body_sha1')['hostname'].nunique().sort_values().reset_index()

In [32]:
http_domain_count[
    http_domain_count['hostname'] > 2
]

,http_response_body_sha1,hostname
347,f56946ffc34566b2af655077422de15d71859967,3
348,60b530b9b7220304daa6244293f746977d7f75b8,4
349,4a6f50923e916965cc72c6e8df7a641993028545,5
350,839f8daa2f5d247204c0dd48686d2a8e877b8721,18
351,983720b96222a9a696a243a6576dbcb86542310b,27
352,c6acd6b7401c703413c1d50799806a973ea19837,31
353,3e3ccd027a49db4b4efbe63054155033788239b5,32
354,489d21c5ef4a7fa528cbcf55904e1f8bcb887453,38


In [33]:
http_domain_count[
    http_domain_count['hostname'] > 10
]

,http_response_body_sha1,hostname
350,839f8daa2f5d247204c0dd48686d2a8e877b8721,18
351,983720b96222a9a696a243a6576dbcb86542310b,27
352,c6acd6b7401c703413c1d50799806a973ea19837,31
353,3e3ccd027a49db4b4efbe63054155033788239b5,32
354,489d21c5ef4a7fa528cbcf55904e1f8bcb887453,38


In [34]:
http_suspicious = http_domain_count[http_domain_count['hostname'] > 10]['http_response_body_sha1'].values.tolist()
http_suspicious


['839f8daa2f5d247204c0dd48686d2a8e877b8721',
 '983720b96222a9a696a243a6576dbcb86542310b',
 'c6acd6b7401c703413c1d50799806a973ea19837',
 '3e3ccd027a49db4b4efbe63054155033788239b5',
 '489d21c5ef4a7fa528cbcf55904e1f8bcb887453']

In [35]:
df_http_answer[df_http_answer['http_response_body_sha1'].isin(http_suspicious)].drop_duplicates(subset=['http_response_body_sha1']).to_csv('2024_IN_http_suspicious.csv')

In [50]:
confirmed_ips = [
    '203.212.192.148','202.164.51.25','49.207.46.62','103.59.214.106','123.176.40.68','106.51.113.17','123.176.40.69','49.207.46.38','123.176.40.67','202.83.21.15','49.205.75.6','202.83.24.75','202.83.21.14', '103.189.63.245'
]

likely_ips = [
    '49.44.79.236','202.83.21.14','218.248.112.60','49.205.171.201','103.206.163.150','49.205.75.11','103.61.253.91','203.109.71.154','36.37.242.94','183.82.14.22']

false_positive_ips = [
    '13.127.247.216','217.146.10.59','139.84.177.246','100.70.19.4','10.197.187.13','10.40.124.13','10.197.186.13','65.20.73.163','208.91.112.55','94.140.14.35','146.112.61.106','178.254.35.97'
]

confirmed_http_resp = [
    '489d21c5ef4a7fa528cbcf55904e1f8bcb887453'
]

likely_http_resp = [
    '3e3ccd027a49db4b4efbe63054155033788239b5'
]

false_positive_http_resp = [
    '644c5ae35cf64adb8a2d3564b57ed0e275265d1f','983720b96222a9a696a243a6576dbcb86542310b','c6acd6b7401c703413c1d50799806a973ea19837','5147c581f12c20ca9274900564cf766635215baf','839f8daa2f5d247204c0dd48686d2a8e877b8721','eb6ce822e8e9989755037c6838f3b100fd6815c3','53b6050219a8a0235b5dd07d50fea304962bb446','fa6441aee67cf31183c3e6477cd5e953df7675f6'
]


In [51]:
%%time
df_with_failure = click_query("""
WITH multiIf(
    dns_failure IS NOT NULL, tuple('dns', dns_failure),
    tcp_failure IS NOT NULL, tuple('tcp', tcp_failure),
    tls_failure IS NOT NULL, tuple('tls', tls_failure),
    http_failure IS NOT NULL, tuple('https', http_failure),
    tuple('ok', '')
) as failure
SELECT 
report_id,
input,
measurement_uid,
probe_cc,
probe_asn,
probe_as_org_name,
probe_as_cc,
network_type,
measurement_start_time,
hostname,
test_name,
ip,
port,
ip_asn,
ip_as_org_name,
resolver_ip,
resolver_cc,
resolver_asn,
resolver_as_org_name,
resolver_as_cc,
dns_engine,
dns_failure,
dns_answer,
tcp_success,
tcp_failure,
tls_is_certificate_valid,
tls_failure,
http_response_body_sha1,
http_request_url,
http_failure,
failure.1 as failure_class,
IF(failure_class = 'ok', 'ok', concat(failure_class, '.', failure_str)) as failure_str_full,
IF(startsWith(failure.2, 'unknown_failure'), 'unknown_failure', failure.2) as failure_str,
failure.2 as failure_str_raw
FROM obs_web
WHERE probe_cc = 'IN' 
AND measurement_start_time > '2024-04-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'web_connectivity'
""")

CPU times: user 41.2 s, sys: 4.1 s, total: 45.3 s
Wall time: 2min 4s


In [52]:
pd.set_option('display.max_colwidth', None)
df_with_failure['explorer_url'] = "https://explorer.ooni.org/m/" + df_with_failure['measurement_uid']

In [53]:
df_with_failure["probe_asn"] = "AS" + df_with_failure["probe_asn"].apply(str)

In [54]:
df_with_failure

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,tcp_success,tcp_failure,tls_is_certificate_valid,tls_failure,http_response_body_sha1,http_request_url,http_failure,failure_class,failure_str_full,failure_str,failure_str_raw,explorer_url
0,20240401T000455Z_webconnectivity_IN_24309_n1_mt88sqfNmoeZzL2K,https://www.facebook.com/,20240401000458.265234_IN_webconnectivity_6d351b702a66b9e6,IN,AS24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,wifi,2024-04-01 00:04:57,www.facebook.com,web_connectivity,163.70.144.35,443.0,32934.0,"Facebook, Inc.",125.20.246.123,IN,9498,Bharti Airtel Limited,IN,system,None,163.70.144.35,1.0,None,1.0,None,None,None,None,ok,ok,,,https://explorer.ooni.org/m/20240401000458.265234_IN_webconnectivity_6d351b702a66b9e6
1,20240401T000455Z_webconnectivity_IN_24309_n1_mt88sqfNmoeZzL2K,https://www.facebook.com/,20240401000458.265234_IN_webconnectivity_6d351b702a66b9e6,IN,AS24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,wifi,2024-04-01 00:04:57,www.facebook.com,web_connectivity,2a03:2880:f188:181:face:b00c:0:25de,443.0,32934.0,"Facebook, Inc.",125.20.246.123,IN,9498,Bharti Airtel Limited,IN,system,None,2a03:2880:f188:181:face:b00c:0:25de,0.0,network_unreachable,NaN,None,None,None,None,tcp,tcp.network_unreachable,network_unreachable,network_unreachable,https://explorer.ooni.org/m/20240401000458.265234_IN_webconnectivity_6d351b702a66b9e6
2,20240401T000455Z_webconnectivity_IN_24309_n1_mt88sqfNmoeZzL2K,https://www.facebook.com/,20240401000458.265234_IN_webconnectivity_6d351b702a66b9e6,IN,AS24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,wifi,2024-04-01 00:04:57,www.facebook.com,web_connectivity,None,NaN,NaN,None,125.20.246.123,IN,9498,Bharti Airtel Limited,IN,None,None,None,NaN,None,NaN,None,8a6c68513ac82c43ba2f10b8f821d7e5b1f60d1d,https://www.facebook.com/,None,ok,ok,,,https://explorer.ooni.org/m/20240401000458.265234_IN_webconnectivity_6d351b702a66b9e6
3,20240401T000455Z_webconnectivity_IN_24309_n1_mt88sqfNmoeZzL2K,https://twitter.com/,20240401000501.878055_IN_webconnectivity_3d03fc0dc28f67e9,IN,AS24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,wifi,2024-04-01 00:05:00,twitter.com,web_connectivity,104.244.42.193,443.0,13414.0,Twitter Inc.,125.20.246.123,IN,9498,Bharti Airtel Limited,IN,system,None,104.244.42.193,1.0,None,1.0,None,None,None,None,ok,ok,,,https://explorer.ooni.org/m/20240401000501.878055_IN_webconnectivity_3d03fc0dc28f67e9
4,20240401T000455Z_webconnectivity_IN_24309_n1_mt88sqfNmoeZzL2K,https://twitter.com/,20240401000501.878055_IN_webconnectivity_3d03fc0dc28f67e9,IN,AS24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,wifi,2024-04-01 00:05:00,twitter.com,web_connectivity,None,NaN,NaN,None,125.20.246.123,IN,9498,Bharti Airtel Limited,IN,None,None,None,NaN,None,NaN,None,b80bb1a6fec1ab5062850ef194d6b076263a5f4a,https://twitter.com/,None,ok,ok,,,https://explorer.ooni.org/m/20240401000501.878055_IN_webconnectivity_3d03fc0dc28f67e9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1863444,20240425T115643Z_webconnectivity_IN_24309_n1_rRHGuJvaBc6VUFGb,https://www.macfound.org/,20240425120338.711343_IN_webconnectivity_6813b185bee44c6e,IN,AS24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,unknown,2024-04-25 12:03:38,www.macfound.org,web_connectivity,104.20.69.80,443.0,13335.0,Cloudflare Inc,49.205.72.130,IN,24309,"Atria Convergence Technologies Pvt. Ltd.,",IN,system,None,104.20.69.80,1.0,None,1.0,None,None,None,None,ok,ok,,,https://explorer.ooni.org/m/20240425120338.711343_IN_webconnectivity_6813b185bee44c6e
1863445,20240425T115643Z_webconnectivity_IN_24309_n1_rRHGuJvaBc6VUFGb,https://www.macfound.org/,20240425120338.711343_IN_webconnectivity_6813b185bee44c6e,IN,AS24309,"Atria 

In [ ]:
#df_with_failure = pd.read_csv('2024-IN-df_with_failure.csv')

In [55]:
len(df_with_failure)

1863449

In [56]:
list(df_with_failure)

['report_id',
 'input',
 'measurement_uid',
 'probe_cc',
 'probe_asn',
 'probe_as_org_name',
 'probe_as_cc',
 'network_type',
 'measurement_start_time',
 'hostname',
 'test_name',
 'ip',
 'port',
 'ip_asn',
 'ip_as_org_name',
 'resolver_ip',
 'resolver_cc',
 'resolver_asn',
 'resolver_as_org_name',
 'resolver_as_cc',
 'dns_engine',
 'dns_failure',
 'dns_answer',
 'tcp_success',
 'tcp_failure',
 'tls_is_certificate_valid',
 'tls_failure',
 'http_response_body_sha1',
 'http_request_url',
 'http_failure',
 'failure_class',
 'failure_str_full',
 'failure_str',
 'failure_str_raw',
 'explorer_url']

In [73]:
# see: https://learn.microsoft.com/en-us/windows/win32/winsock/windows-sockets-error-codes-2
unknown_failure_map = {
    ': server misbehaving': 'dns_server_misbehaving',
    ': read: connection refused': 'connection_refused',
    ': connect: network is unreachable': 'network_unreachable',
    'tls: first record does not look like a TLS handshake': 'tls_bad_first_record',
    'remote error: tls: handshake failure': 'tls_handshake_failure',
    'remote error: tls: illegal parameter': 'tls_illegal_parameter',
    'connectex: No connection could be made because the target machine actively refused it': 'connection_refused',
    'read: connection refused': 'connection_refused',
    'remote error: tls: access denied': 'tls_access_denied',
    'remote error: tls: internal error': 'tls_internal_error',
    'HTTP/1.x transport connection broken: malformed HTTP version': 'http_malformed_response',
    'net/http: timeout awaiting response headers': 'http_timeout',
    'read: operation timed out': 'timed_out',
    'connect: operation timed out': 'timed_out',
    ': No address associated with hostname': 'dns_nxdomain_error',
    ': connect: bad file descriptor': 'bad_file_descriptor',
    'stream error: stream ID': 'http_stream_error',
    
    # This looks more like a golang-bug: https://github.com/golang/go/issues/31259
    'readLoopPeekFailLocked: <nil>': 'http_golang_bug',

    ': connect: no route to host': 'host_unreachable',
    ': connect: cannot assign requested address': 'address_not_available',
    'getaddrinfow: The requested name is valid, but no data of the requested type was found.': 'dns_no_answer',
    'wsarecv: Se ha forzado la interrupción de una conexión existente por el host remoto.': 'connection_reset',
    'wsarecv: An existing connection was forcibly closed by the remote host.': 'connection_reset',
    'wsarecv: Connessione in corso interrotta forzatamente dall\'host remoto.': 'connection_reset',
    'wsarecv: Uma ligação existente foi forçada a fechar pelo anfitrião remoto': 'connection_reset',
    
    'getaddrinfow: Ceci est habituellement une erreur temporaire qui se produit durant la résolution du nom d’hôte et qui signifie que le serveur local n’a pas reçu de réponse d’un serveur faisant autorité': 'dns_temporary_failure',
    'getaddrinfow: Dies ist normalerweise ein zeitweiliger Fehler bei der Auflösung von Hostnamen. Grund ist, dass der lokale Server keine Rückmeldung vom autorisierenden Server erhalten hat.': 'dns_temporary_failure',
    'getaddrinfow: Este é geralmente um erro temporário durante a resolução de nomes de anfitrião e significa que o servidor local não recebeu uma resposta de um servidor autoritário': 'dns_temporary_failure',
    'getaddrinfow: Éste es normalmente un error temporal durante la resolución de nombres de host y significa que el servidor local no recibió una respuesta de un servidor autoritativo': 'dns_temporary_failure',
}
def map_unknown_failure(failure_str):
    if not failure_str.startswith("unknown_failure"):
        return failure_str
    for substring, clean_failure in unknown_failure_map.items():
        if substring in failure_str:
            return clean_failure
    return "unknown_failure"

def simplify_failure(failure_str):
    if failure_str in ['timed_out', 'generic_timeout_error', 'deferred_timeout_error']:
        return 'timeout'
    
    if failure_str in ['android_dns_cache_no_data', 'dns_nxdomain_error']:
        return 'nxdomain'
    
    if failure_str in ['connection_refused', 'connection_refused_error']:
        return 'connection_refused'
        
    return failure_str

ipv6_failures = ['address_not_available', 'address_family_not_supported', 'network_unreachable', 'host_unreachable']
def compute_analysis(row):
    failure_str = map_unknown_failure(row['failure_str_raw'])

    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in confirmed_ips:
        return 'dns.confirmed'
    
    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in likely_ips:
        return 'dns.likely.blocked'
    
    if not pd.isnull(row['dns_answer']) and row['dns_answer'] in false_positive_ips:
        return 'ok'
        
    if row['tls_is_certificate_valid'] == True:
        return 'ok'

    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in confirmed_http_resp:
        return 'http.confirmed'
    
    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in likely_http_resp:
        return 'http.likely.blocked'
    
    if not pd.isnull(row['http_request_url']) and row['http_response_body_sha1'] in false_positive_http_resp:
        return 'ok'
    
    if row['failure_class'] == 'ok':
        return 'ok'

    if row['dns_consistency'] == 'inconsistent':
        return 'dns.inconsistent'
    
    if row['ip_as_org_name'] == 'Bogon':
        return 'dns.bogon'
    
    #if row['dns_blocking_scope'] not in ('u', 'n'):
    #    return f"dns.{row['dns_blocking_scope']}"
    
    if row['tls_is_certificate_valid'] == False:
        return 'tls.bad_cert'
    
    simple_failure = simplify_failure(failure_str)
    if simple_failure in ipv6_failures and row['ip'] and ':' in row['ip']:
        return 'ipv6_error'

    if simple_failure.startswith("ssl_"):
        simple_failure = 'bad_cert'
    
    prefix = row['failure_class']
    if prefix == 'https' and simple_failure.startswith('dns_') or simple_failure == 'nxdomain':
        prefix = 'dns'

    return f'{prefix}.{simple_failure}'

In [58]:
df_ctrl_dns_obs_web = click_query("""
SELECT 
groupUniqArray(ip) as uniq_ip,
groupUniqArray(ip_asn) as uniq_asn,
groupUniqArray(ip_as_org_name) as uniq_as_org_name,
hostname
FROM obs_web
WHERE measurement_start_time > %(measurement_start_day)s
AND probe_cc NOT IN %(cc_list)s
AND tls_is_certificate_valid = 1 
GROUP BY hostname
""", **{
    "measurement_start_day": MEASUREMENT_START_DAY,
    "cc_list": ANALYSIS_COUNTRY_CODES,
})

In [59]:
%%time
df_ctrl_dns_obs_web_ctrl = click_query("""
SELECT 
groupUniqArray(ip) as uniq_ip,
groupUniqArray(ip_asn) as uniq_asn,
groupUniqArray(ip_as_org_name) as uniq_as_org_name,
hostname
FROM obs_web_ctrl
WHERE measurement_start_time > %(measurement_start_day)s
AND tls_success = 1 
GROUP BY hostname
""", **{
    "measurement_start_day": MEASUREMENT_START_DAY
})

CPU times: user 743 ms, sys: 90.7 ms, total: 834 ms
Wall time: 31.1 s


In [60]:
from collections import defaultdict
ctrl_map = defaultdict(dict)
for _, row in df_ctrl_dns_obs_web_ctrl.iterrows():
    ctrl_map[row['hostname']]['uniq_ip_ctrl'] = set(row['uniq_ip'])
    ctrl_map[row['hostname']]['uniq_asn_ctrl'] = set(row['uniq_asn'])
    ctrl_map[row['hostname']]['uniq_as_org_name_ctrl'] = set(row['uniq_as_org_name'])
    
for _, row in df_ctrl_dns_obs_web.iterrows():
    ctrl_map[row['hostname']]['uniq_ip_web'] = set(row['uniq_ip'])
    ctrl_map[row['hostname']]['uniq_asn_web'] = set(row['uniq_asn'])
    ctrl_map[row['hostname']]['uniq_as_org_name_web'] = set(row['uniq_as_org_name'])

In [61]:
df_ctrl_dns_obs_web.head()

,uniq_ip,uniq_asn,uniq_as_org_name,hostname
0,[2a06:98c1:3121::3],[13335],[Cloudflare Inc],desk.im
1,"[46.107.16.55, 46.107.16.54, 2001:4c48:16:6::1:20, 2001:4c48:16:6::2:20]",[5483],[Magyar Telekom plc.],videa.hu
2,[185.34.219.59],[198203],[RouteLabel V.O.F.],women.jo
3,"[172.67.179.43, 104.21.31.185]",[13335],[Cloudflare Inc],es.zenit.org
4,"[2606:50c0:8002::153, 185.199.110.153, 185.199.109.153, 2606:50c0:8001::153, 185.199.108.153, 185.199.111.153, 2606:50c0:8000::153, 2606:50c0:8003::153]",[54113],"[Fastly, Inc.]",sibrea.github.io


In [62]:
df_ctrl_dns_obs_web_ctrl.head()

,uniq_ip,uniq_asn,uniq_as_org_name,hostname
0,"[188.114.97.0, 2606:4700:3036::ac43:a8d1, 188.114.96.3, 172.67.168.209, 2a06:98c1:3121::9, 2606:4700:3036::6815:2eb2, 188.114.97.4, 104.21.46.178, 2a06:98c1:3121::3, 2a06:98c1:3121::, 188.114.96.9, 2a06:98c1:3120::4, 2a06:98c1:3121::4, 188.114.97.9, 2a06:98c1:3120::, 2a06:98c1:3120::3, 188.114.96.4, 2a06:98c1:3120::9, 188.114.97.3, 188.114.96.0]",[13335],[Cloudflare Inc],ishr.ch
1,"[104.21.24.126, 2606:4700:3032::ac43:dacb, 172.67.218.203, 2606:4700:3031::6815:187e]",[13335],[Cloudflare Inc],ppy.buzz
2,[31.133.222.2],[50245],[Serverel Inc.],fpo.xxx
3,"[23.50.131.148, 2a02:26f0:c900:b::5f65:4a03, 23.53.43.17, 2a02:26f0:b200::58dd:1933, 95.101.111.141, 2.20.143.81, 23.50.131.154, 23.36.162.196, 2a02:26f0:3500:14::1724:a249, 2.16.6.11, 2a02:26f0:ab00::5c7a:d728, 2.18.244.74, 2a02:26f0:b200::58dd:193a, 2.20.143.97, 23.36.162.73, 95.101.111.157, 2a02:26f0:ab00::214:8e4a, 2a02:26f0:c900:b::5f65:4a18, 2.18.244.96, 23.53.43.74, 2a02:26f0:3500:14::1724:a258, 104.110.191.13, 23.36.162.216, 2a02:26f0:1180:33::210:64c, 104.110.191.46, 2.16.6.23, 2a02:26f0:1180:33::210:64f, 23.36.162.88]",[20940],[Akamai International B.V.],trouw.nl
4,"[104.16.41.196, 13.49.232.30, 2606:4700::6810:2cc4, 2606:4700::6810:2ac4, 2606:4700::6810:2dc4, 104.16.45.196, 104.16.42.196, 104.16.43.196, 13.48.174.154, 2606:4700::6810:2bc4, 104.16.44.196, 2606:4700::6810:29c4]","[13335, 16509]","[Cloudflare Inc, Amazon.com, Inc.]",www.redporn.com


In [63]:
def compute_dns_consistency(row):
    if pd.isnull(row['ip']):
        return None
    ctrl = ctrl_map[row['hostname']]
    if len(ctrl):
        return 'unknown'
    if row['ip'] in ctrl.get('uniq_ip_ctrl', {}):        
        return 'consistent.ip_ctrl'
    if row['ip'] in ctrl.get('uniq_ip_web', {}):
        return 'consistent.ip_web'
    if row['ip_asn'] in ctrl.get('uniq_asn_ctrl', {}):
        return 'consistent.asn_ctrl'
    if row['ip_asn'] in ctrl.get('uniq_asn_web', {}):
        return 'consistent.asn_web'
    if row['ip_as_org_name'] in ctrl.get('uniq_as_org_name_ctrl', {}):
        return 'consistent.as_org_name_ctrl'
    if row['ip_as_org_name'] in ctrl.get('uniq_as_org_name_web', {}):
        return 'consistent.as_org_name_web'
    return 'inconsistent'

In [64]:
df_with_failure['dns_consistency'] = df_with_failure.progress_apply(compute_dns_consistency, axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1863449/1863449 [00:16<00:00, 109690.00it/s]


In [65]:
df_with_failure['analysis'] = df_with_failure.progress_apply(compute_analysis, axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1863449/1863449 [00:47<00:00, 39262.36it/s]


In [66]:
df_with_failure.to_csv('2024-IN-df_with_failure.csv')

In [67]:
confirmed_blocking = df_with_failure[
    (df_with_failure['analysis'] == 'dns.confirmed') | 
    (df_with_failure['analysis'] == 'http.confirmed')]

msmt_counts_confirmed = confirmed_blocking.pivot_table(index=['input','probe_cc'], values=['measurement_uid', 'analysis', 'probe_asn'],
                                                aggfunc={'measurement_uid': 'nunique', 
                                                         'analysis': lambda x: ', '.join(set(x)),
                                                         'probe_asn': lambda x: ', '.join(set(x))}).reset_index().rename(columns = {
                                                                    'measurement_uid': 'blocking_count',
                                                                    'analysis' : 'blocking_method',
                                                                    'probe_asn' : 'probe_asn_blocking'})

In [68]:
likely_blocking = df_with_failure[
    (df_with_failure['analysis'] == 'dns.likely.blocked') |
    (df_with_failure['analysis'] == 'http.likely.blocked')]

msmt_counts_likely = likely_blocking.pivot_table(index=['input','probe_cc'], values=['measurement_uid', 'analysis', 'probe_asn'],
                                                aggfunc={'measurement_uid': 'nunique', 
                                                         'analysis': lambda x: ', '.join(set(x)),
                                                         'probe_asn': lambda x: ', '.join(set(x))}).reset_index().rename(columns = {
                                                                    'measurement_uid': 'blocking_count',
                                                                    'analysis' : 'blocking_method',
                                                                    'probe_asn' : 'probe_asn_blocking'})

In [69]:
msmt_counts_confirmed.to_csv('2024-india-confirmed-blocked-heuristics.csv')

In [70]:
msmt_counts_likely.to_csv('2024-india-likely-blocked-heuristics.csv')

In [102]:
df_with_failure[
    df_with_failure['failure_str_raw'].str.startswith('unknown_failure')
][['failure_str_raw', 'analysis']].drop_duplicates()

,failure_str_raw,analysis
2079250,unknown_failure: connection error: PROTOCOL_ERROR,https.unknown_failure
2936041,unknown_failure: http2: timeout awaiting response headers,https.unknown_failure
4486740,unknown_failure: write tcp [scrubbed]->[scrubbed]: write: broken pipe,tls.unknown_failure
4764350,unknown_failure: write tcp [scrubbed]->[scrubbed]: write: broken pipe,https.unknown_failure
4886103,unknown_failure: remote error: tls: protocol version not supported,https.unknown_failure
5589254,unknown_failure: tls: server selected unsupported protocol version 301,dns.inconsistent
8581295,unknown_failure: remote error: tls: error decoding message,tls.unknown_failure


In [103]:
df_with_failure[
    df_with_failure['ip_as_org_name'] == 'Bogon'
]

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,tcp_success,tcp_failure,tls_handshake_time,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_success,tls_is_certificate_valid,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_certificate_chain_length,tls_failure,http_request_url,http_failure,http_runtime,failure_class,failure_str_full,failure_str,failure_str_raw,dns_consistency,analysis
1547,20230712T092917Z_webconnectivity_VN_18403_n1_tyMXuHYsWsEfDXD5,https://tambao.info/amp/,20230712094113.888004_VN_webconnectivity_6d308ad541d14711,VN,18403,FPT Telecom Company,VN,unknown,2023-07-12 09:41:13,tambao.info,web_connectivity,127.0.0.1,443.0,64666.0,Bogon,210.245.24.101,VN,18403,FPT Telecom Company,VN,system,None,127.0.0.1,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.connection_refused,connection_refused,connection_refused,unknown,dns.bogon
1608,20230712T092917Z_webconnectivity_VN_18403_n1_tyMXuHYsWsEfDXD5,https://the88project.org/,20230712094147.204673_VN_webconnectivity_686f917155123bf0,VN,18403,FPT Telecom Company,VN,unknown,2023-07-12 09:41:47,the88project.org,web_connectivity,::1,443.0,64666.0,Bogon,210.245.24.101,VN,18403,FPT Telecom Company,VN,system,None,::1,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.connection_refused,connection_refused,connection_refused,unknown,dns.bogon
1620,20230712T092917Z_webconnectivity_VN_18403_n1_tyMXuHYsWsEfDXD5,https://thoibao.com/,20230712094159.885458_VN_webconnectivity_cb4e963b348de059,VN,18403,FPT Telecom Company,VN,unknown,2023-07-12 09:41:59,thoibao.com,web_connectivity,::1,443.0,64666.0,Bogon,210.245.24.101,VN,18403,FPT Telecom Company,VN,system,None,::1,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.connection_refused,connection_refused,connection_refused,unknown,dns.bogon
1632,20230712T092917Z_webconnectivity_VN_18403_n1_tyMXuHYsWsEfDXD5,https://tiengnoidanchu.wordpress.com/,20230712094231.778810_VN_webconnectivity_af87e22bc6b77a1c,VN,18403,FPT Telecom Company,VN,unknown,2023-07-12 09:42:31,tiengnoidanchu.wordpress.com,web_connectivity,127.0.0.1,443.0,64666.0,Bogon,210.245.24.101,VN,18403,FPT Telecom Company,VN,system,None,127.0.0.1,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.connection_refused,connection_refused,connection_refused,unknown,dns.bogon
1633,20230712T092917Z_webconnectivity_VN_18403_n1_tyMXuHYsWsEfDXD5,https://tiengnoidanchu.wordpress.com/,20230712094231.778810_VN_webconnectivity_af87e22bc6b77a1c,VN,18403,FPT Telecom Company,VN,unknown,2023-07-12 09:42:31,tiengnoidanchu.wordpress.com,web_connectivity,::1,443.0,64666.0,Bogon,210.245.24.101,VN,18403,FPT Telecom Company,VN,system,None,::1,0.0,connection_refused,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.connection_refused,connection_refused,connection_refused,unknown,dns.bogon
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10066206,20231109T193747Z_webconnectivity_VN_7552_n1_Uko6l2SKhRlC90Gk,https://baotiengdan.com/,20231109193836.752635_VN_webconnectivity_ddb1f4b090678c86,VN,7552,Viettel Group,VN,wifi,

In [104]:
df_with_failure[
    df_with_failure['analysis'] == 'dns.confirmed'
]

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,tcp_success,tcp_failure,tls_handshake_time,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_success,tls_is_certificate_valid,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_certificate_chain_length,tls_failure,http_request_url,http_failure,http_runtime,failure_class,failure_str_full,failure_str,failure_str_raw,dns_consistency,analysis
7024,20240302T031115Z_webconnectivity_VN_7552_n1_Y6YFgIgmts0VsR1V,http://www.tqlcvn.org/,20240302031322.815226_VN_webconnectivity_a92b08d7e018fcf0,VN,7552,Viettel Group,VN,mobile,2024-03-02 03:13:23,www.tqlcvn.org,web_connectivity,125.235.4.59,80.0,7552.0,Viettel Group,27.71.194.194,VN,7552,Viettel Group,VN,system,None,125.235.4.59,0.0,generic_timeout_error,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.generic_timeout_error,generic_timeout_error,generic_timeout_error,inconsistent,dns.confirmed
8678,20240302T033226Z_webconnectivity_VN_7552_n1_Xet7gZfkWNWS5xju,https://scontent-frt3-2.cdninstagram.com/favicon.ico,20240302033631.268289_VN_webconnectivity_23790dba62193cec,VN,7552,Viettel Group,VN,wifi,2024-03-02 03:36:30,scontent-frt3-2.cdninstagram.com,web_connectivity,125.235.4.59,443.0,7552.0,Viettel Group,27.68.251.238,VN,7552,Viettel Group,VN,system,None,125.235.4.59,0.0,generic_timeout_error,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,tcp,tcp.generic_timeout_error,generic_timeout_error,generic_timeout_error,inconsistent,dns.confirmed
9213,20240302T033226Z_webconnectivity_VN_7552_n1_Xet7gZfkWNWS5xju,https://www.ms88vtv.com/,20240302033845.448078_VN_webconnectivity_b4dba8dbaf3748ce,VN,7552,Viettel Group,VN,wifi,2024-03-02 03:38:44,www.ms88vtv.com,web_connectivity,125.212.193.174,443.0,38731.0,Vietel - CHT Compamy Ltd,27.68.251.238,VN,7552,Viettel Group,VN,system,None,125.212.193.174,1.0,None,0.048580,6.0,2.0,12528.0,288.0,write_2,1,0.0,"1.2.840.113549.1.9.1=webmaster@localhost,CN=localhost,OU=none,O=none,L=Sometown,ST=Someprovince,C=US",localhost,"1.2.840.113549.1.9.1=webmaster@localhost,CN=localhost,OU=none,O=none,L=Sometown,ST=Someprovince,C=US",localhost,[],2049-03-08 06:12:46,2021-10-22 06:12:46,1.0,ssl_invalid_hostname,None,None,NaN,tls,tls.ssl_invalid_hostname,ssl_invalid_hostname,ssl_invalid_hostname,unknown,dns.confirmed
9888,20240302T033226Z_webconnectivity_VN_7552_n1_Xet7gZfkWNWS5xju,https://www.vuicungdafa.com/vn/,20240302034111.762361_VN_webconnectivity_3dd4c9c34d1b6218,VN,7552,Viettel Group,VN,wifi,2024-03-02 03:41:10,www.vuicungdafa.com,web_connectivity,125.212.193.174,443.0,38731.0,Vietel - CHT Compamy Ltd,27.68.251.238,VN,7552,Viettel Group,VN,system,None,125.212.193.174,1.0,None,0.047289,4.0,2.0,12528.0,292.0,write_2,1,0.0,"1.2.840.113549.1.9.1=webmaster@localhost,CN=localhost,OU=none,O=none,L=Sometown,ST=Someprovince,C=US",localhost,"1.2.840.113549.1.9.1=webmaster@localhost,CN=localhost,OU=none,O=none,L=Sometown,ST=Someprovince,C=US",localhost,[],2049-03-08 06:12:46,2021-10-22 06:12:46,1.0,ssl_invalid_hostname,None,None,NaN,tls,tls.ssl_invalid_hostname,ssl_invalid_hostname,ssl_invalid_hostname,inconsistent,dns.confirmed
58002,20240302T062617Z_webconnectivity_VN_7552_n1_1O3DZ0IQ9xVrd57J,https://vnhrdmay11.org/,20240302062817.607006_VN_webconnectivity_02c6299da4f1f3f0,VN,7552,Viettel Group,VN,wifi,2024-03-02 06:28:15,vnhrdmay11.org,web_connectivity,125.235.4.59,443.0,7552.0,Viettel Group,27.71.195.66,VN,75

In [105]:
len(df_with_failure['failure_str_full'].unique())

54

In [106]:
len(df_with_failure['analysis'].unique())

45

In [107]:
df_with_failure[
    (df_with_failure['probe_cc'] == 'VN')
    & (df_with_failure['analysis'] != 'ipv6_error')
]

,report_id,input,measurement_uid,probe_cc,probe_asn,probe_as_org_name,probe_as_cc,network_type,measurement_start_time,hostname,test_name,ip,port,ip_asn,ip_as_org_name,resolver_ip,resolver_cc,resolver_asn,resolver_as_org_name,resolver_as_cc,dns_engine,dns_failure,dns_answer,tcp_success,tcp_failure,tls_handshake_time,tls_handshake_read_count,tls_handshake_write_count,tls_handshake_read_bytes,tls_handshake_write_bytes,tls_handshake_last_operation,tls_success,tls_is_certificate_valid,tls_end_entity_certificate_subject,tls_end_entity_certificate_subject_common_name,tls_end_entity_certificate_issuer,tls_end_entity_certificate_issuer_common_name,tls_end_entity_certificate_san_list,tls_end_entity_certificate_not_valid_after,tls_end_entity_certificate_not_valid_before,tls_certificate_chain_length,tls_failure,http_request_url,http_failure,http_runtime,failure_class,failure_str_full,failure_str,failure_str_raw,dns_consistency,analysis
0,20240109T235539Z_webconnectivity_VN_7552_n1_aQEJI8CYCwIlhVtW,http://www.gayscape.com/,20240110000005.868806_VN_webconnectivity_803068196f56d4df,VN,7552,Viettel Group,VN,wifi,2024-01-10 00:00:02,www.gayscape.com,web_connectivity,204.8.234.161,80.0,395412.0,VS Media Inc.,172.253.4.2,HK,15169,Google LLC,US,system,None,204.8.234.161,1.0,None,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,None,None,NaN,ok,ok,,,unknown,ok
1,20240109T235539Z_webconnectivity_VN_7552_n1_aQEJI8CYCwIlhVtW,http://www.gayscape.com/,20240110000005.868806_VN_webconnectivity_803068196f56d4df,VN,7552,Viettel Group,VN,wifi,2024-01-10 00:00:02,www.gayscape.com,web_connectivity,None,NaN,NaN,None,172.253.4.2,HK,15169,Google LLC,US,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,https://www.gayscape.com/,None,NaN,ok,ok,,,None,ok
2,20240109T235539Z_webconnectivity_VN_7552_n1_aQEJI8CYCwIlhVtW,http://www.gayscape.com/,20240110000005.868806_VN_webconnectivity_803068196f56d4df,VN,7552,Viettel Group,VN,wifi,2024-01-10 00:00:02,www.gayscape.com,web_connectivity,None,NaN,NaN,None,172.253.4.2,HK,15169,Google LLC,US,None,None,None,NaN,None,NaN,NaN,NaN,NaN,NaN,None,0,NaN,None,None,None,None,[],NaT,NaT,NaN,None,http://www.gayscape.com/,None,NaN,ok,ok,,,None,ok
3,20240109T235539Z_webconnectivity_VN_7552_n1_aQEJI8CYCwIlhVtW,https://www.gaystarnews.com/,20240110000006.989721_VN_webconnectivity_8f2e27c327ff67a5,VN,7552,Viettel Group,VN,wifi,2024-01-10 00:00:06,www.gaystarnews.com,web_connectivity,104.21.234.125,443.0,13335.0,Cloudflare Inc,172.253.4.2,HK,15169,Google LLC,US,system,None,104.21.234.125,1.0,None,0.085150,2.0,4.0,5552.0,397.0,write_4,1,1.0,CN=gaystarnews.com,gaystarnews.com,"CN=GTS CA 1P5,O=Google Trust Services LLC,C=US",GTS CA 1P5,"[gaystarnews.com, *.gaystarnews.com]",2024-04-07 08:26:03,2024-01-08 08:26:04,3.0,None,None,None,NaN,ok,ok,,,unknown,ok
4,20240109T235539Z_webconnectivity_VN_7552_n1_aQEJI8CYCwIlhVtW,https://www.gaystarnews.com/,20240110000006.989721_VN_webconnectivity_8f2e27c327ff67a5,VN,7552,Viettel Group,VN,wifi,2024-01-10 00:00:06,www.gaystarnews.com,web_connectivity,104.21.234.124,443.0,13335.0,Cloudflare Inc,172.253.4.2,HK,15169,Google LLC,US,system,None,104.21.234.124,1.0,None,0.093481,2.0,4.0,5552.0,397.0,write_4,1,1.0,CN=gaystarnews.com,gaystarnews.com,"CN=GTS CA 1P5,O=Google Trust Services LLC,C=US",GTS CA 1P5,"[gaystarnews.com, *.gaystarnews.com]",2024-04-07 08:26:03,2024-01-08 08:26:04,3.0,None,None,None,NaN,ok,ok,,,unknown,ok
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10069620,20231109T203520Z_webconnectivity_VN_133606_n1_QLEbDWGC6LrxTnox,https://www.vice.com/,20231109210122.902658_VN_webconnectivity_397a21ea447095d8,VN,133606,Viettel Timor Leste,TL,wifi,2023-11-09 21:01:12,www.vice.com,web_connectivity,151.101.66.133,443.0,54113.0,"Fastly, Inc.",172.253.215.197,BE,15169,Google LLC,US,syst

In [71]:
df_with_failure[
     (df_with_failure['analysis'] != 'ok')
]['analysis'].unique()

array(['ipv6_error', 'dns.confirmed', 'unknown_error', 'dns.inconsistent',
       'tls.connection_reset', 'dns.likely.blocked', 'tcp.timeout',
       'http.confirmed', 'http.likely.blocked', 'tls.timeout',
       'tls.bad_cert', 'tcp.connection_refused',
       'tls.tls_handshake_failure', 'dns.bogon', 'tls.eof_error',
       'tls.invalid_record', 'tcp.host_unreachable',
       'tls.connection_aborted', 'tls.internal_error',
       'tcp.connection_reset', 'tcp.connection_aborted',
       'tls.unknown_failure'], dtype=object)

In [72]:
%%time
target_domains = [
    "www.tiktok.com"
]
df_special = click_query("""
WITH multiIf(
    dns_failure IS NOT NULL, tuple('dns', dns_failure),
    tcp_failure IS NOT NULL, tuple('tcp', tcp_failure),
    tls_failure IS NOT NULL, tuple('tls', tls_failure),
    http_failure IS NOT NULL, tuple('https', http_failure),
    tuple('ok', '')
) as failure
SELECT 
report_id,
input,
measurement_uid,
probe_cc,
probe_asn,
probe_as_org_name,
probe_as_cc,
network_type,
measurement_start_time,
hostname,
test_name,
ip,
port,
ip_asn,
ip_as_org_name,
resolver_ip,
resolver_cc,
resolver_asn,
resolver_as_org_name,
resolver_as_cc,
dns_engine,
dns_failure,
dns_answer,
tcp_success,
tcp_failure,
tls_handshake_time,
tls_handshake_read_count,
tls_handshake_write_count,
tls_handshake_read_bytes,
tls_handshake_write_bytes,
tls_handshake_last_operation,
tls_cipher_suite IS NOT NULL as tls_success,
tls_is_certificate_valid,
tls_end_entity_certificate_subject,
tls_end_entity_certificate_subject_common_name,
tls_end_entity_certificate_issuer,
tls_end_entity_certificate_issuer_common_name,
tls_end_entity_certificate_san_list,
tls_end_entity_certificate_not_valid_after,
tls_end_entity_certificate_not_valid_before,
tls_certificate_chain_length,
tls_failure,
http_response_body_sha1,
http_request_url,
http_failure,
failure.1 as failure_class,
IF(failure_class = 'ok', 'ok', concat(failure_class, '.', failure_str)) as failure_str_full,
IF(startsWith(failure.2, 'unknown_failure'), 'unknown_failure', failure.2) as failure_str,
failure.2 as failure_str_raw
FROM obs_web
WHERE probe_cc = 'IN' 
AND measurement_start_time > '2023-07-01'
AND measurement_start_time < '2024-07-01'
AND test_name = 'web_connectivity'
AND hostname IN %(target_domains)s
""", target_domains=target_domains)

CPU times: user 10.1 s, sys: 723 ms, total: 10.8 s
Wall time: 1min 47s


In [74]:
len(df_special)

27304

In [75]:
df_special['dns_consistency'] = df_special.progress_apply(compute_dns_consistency, axis=1)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27304/27304 [00:00<00:00, 110842.31it/s]


In [76]:
df_special['analysis'] = df_special.progress_apply(compute_analysis, axis=1)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 27304/27304 [00:00<00:00, 41958.35it/s]


In [77]:
df_special["probe_asn"] = "AS" + df_special["probe_asn"].apply(str)

In [80]:
method_of_blocking = df_special[df_special['analysis'] != 'ok'].pivot_table(index=['analysis'], values=['measurement_uid'], columns=['probe_asn','probe_as_org_name'],
                                                aggfunc={'measurement_uid': len}).fillna(0).reset_index()

method_of_blocking

analysis      measurement_uid  \
probe_asn                                                AS131269   
probe_as_org_name                            Beam Telecom Pvt Ltd   
0                                  dns.bogon                  0.0   
1                              dns.confirmed                  0.0   
2                          dns.dns_no_answer                  0.0   
3                         dns.likely.blocked                 26.0   
4                               dns.nxdomain                  0.0   
5                                dns.timeout                  0.0   
6                        dns.unknown_failure                  0.0   
7                             https.bad_cert                  3.0   
8                  https.bad_file_descriptor                  0.0   
9                   https.connection_refused                  0.0   
10                    https.connection_reset                  0.0   
11                           https.eof_error                 26.0   
12                      https.invalid_record                  0.0   
13                             https.timeout                  0.0   
14                     https.unknown_failure                  0.0   
15                                ipv6_error                  0.0   
16                   tcp.bad_file_descriptor                  0.0   
17                      tcp.host_unreachable                  0.0   
18                               tcp.timeout                  0.0   
19                       tcp.unknown_failure                  0.0   
20                              tls.bad_cert                  6.0   
21                      tls.connection_reset                  0.0   
22                             tls.eof_error                  0.0   
23                        tls.internal_error                  0.0   
24                        tls.invalid_record                  0.0   
25                               tls.timeout                  0.0   

                                                                  \
probe_asn               AS131473                        AS132115   
probe_as_org_name SRM University INDINET SERVICE PRIVATE LIMITED   
0                            0.0                             0.0   
1                            0.0                             0.0   
2                            0.0                             0.0   
3                            0.0                             0.0   
4                            0.0                             0.0   
5                            0.0                             0.0   
6                            0.0                             0.0   
7                            0.0                             0.0   
8                            0.0                             0.0   
9                            0.0                             0.0   
10                           5.0                             0.0   
11                           0.0                             0.0   
12                           0.0                             0.0   
13                           0.0                             0.0   
14                           0.0                             0.0   
15                           0.0                             0.0   
16                           0.0                             0.0   
17                           0.0                             0.0   
18                           0.0                             4.0   
19                           0.0                             0.0   
20                           0.0                             0.0   
21                          41.0                             0.0   
22                           0.0                             0.0   
23                           0.0                             0.0   
24                           0.0                             0.0   
25                           0.0                             0.0   

                                                         \
pr